In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df = pd.read_csv('train_dup.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [19]:
duplicates = df[df.is_duplicate==1]
not_duplicates = df[df.is_duplicate == 0]
print(f'The proportion of classes pairs/not pairs is: {len(duplicates)}/{len(not_duplicates)}')
print(f'An example of duplicated questions: \n{duplicates.question1[5]} \nAND \n{duplicates.question2[5]}')

The proportion of classes pairs/not pairs is: 149263/255027
An example of duplicated questions: 
Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? 
AND 
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


,id,qid1,qid2,question1,question2,is_duplicate
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1


In [31]:
# checking out from habr. This example will be later removed
stop_words = set(stopwords.words("english"))
sentence = duplicates.question1[5]
#print(stopwords.words("english"))
words = nltk.word_tokenize(sentence)
without_stop_words = [word for word in words if not word in stop_words]
print(without_stop_words)

['Astrology', ':', 'I', 'Capricorn', 'Sun', 'Cap', 'moon', 'cap', 'rising', '...', 'say', '?']


In [33]:
# checking out from habr. This example will be later removed
def compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word, pos):
    """
    Print the results of stemmind and lemmitization using the passed stemmer, lemmatizer, word and pos (part of speech)
    """
    print("Stemmer:", stemmer.stem(word))
    print("Lemmatizer:", lemmatizer.lemmatize(word, pos))
    print()

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word = "seen", pos = wordnet.VERB)
compare_stemmer_and_lemmatizer(stemmer, lemmatizer, word = "drove", pos = wordnet.VERB)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maryp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Stemmer: seen
Lemmatizer: see

Stemmer: drove
Lemmatizer: drive



In [37]:
count_vectorizer = CountVectorizer()
print(duplicates.question1[5], duplicates.question2[5])
# Create the Bag-of-Words Model
bag_of_words = count_vectorizer.fit_transform([duplicates.question1[5], duplicates.question2[5]])

# Show the Bag-of-Words Model as a pandas DataFrame
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me? I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


,about,am,and,ascendant,astrology,cap,capricorn,does,in,me,moon,rising,say,sun,that,this,triple,what
0,1,1,1,0,1,2,1,1,0,1,1,1,1,1,1,0,0,1
1,1,0,1,1,0,0,2,1,1,1,1,0,1,1,0,1,1,1


Так, мы здесь взяли 2 похожих вопроса, каждый соответствует ряду теперь. Можно заметить, что вектора типа очень близки, но нужно смотреть на ключевые слова, а не на предлоги, союзы и т.д


Мы будем потом выбрасывать эти стоп-слова, которые есть в stopwords.words("english")

# Планы:

игнорирование регистра слов;


игнорирование пунктуации;


выкидывание стоп-слов;


приведение слов к их базовым формам (лемматизация);


исправление неправильно написанных слов.


подсчет н-грамм


TF/IDF - как помогут?


как преобразовать в числа?

In [45]:
def clean_sentence_stemmer(sentence):
    """
    Receives a raw sentence and clean it using the following steps:
    1. Remove all non-words
    2. Transform the review in lower case
    3. Remove all stop words
    4. Perform stemming

    Args:
        sentence: the sentence that will be cleaned
    Returns:
        a clean sentence using the mentioned steps above.
    """
    
    sentence = re.sub("[^A-Za-z]", " ", sentence)
    sentence = sentence.lower()
    sentence = word_tokenize(sentence)
    stemmer = PorterStemmer()
    sentence = [stemmer.stem(word) for word in sentence if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)
    return sentence

clean_sentence_stemmer(duplicates.question1[5])

'astrolog capricorn sun cap moon cap rise say'

In [46]:
def clean_sentence_lemmatizer(sentence):
    """
    Receives a raw sentence and clean it using the following steps:  # BETTER
    1. Remove all non-words
    2. Transform the review in lower case
    3. Remove all stop words
    4. Perform lemmatizer

    Args:
        sentence: the sentence that will be cleaned
    Returns:
        a clean sentence using the mentioned steps above.
    """
    
    sentence = re.sub("[^A-Za-z]", " ", sentence)
    sentence = sentence.lower()
    sentence = word_tokenize(sentence)
    lemmatizer = WordNetLemmatizer()
    sentence = [lemmatizer.lemmatize(word) for word in sentence if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)
    return sentence

clean_sentencelemm(duplicates.question1[5])

'astrology capricorn sun cap moon cap rising say'

Ну все, попробуем применить последнюю функцию к всей дате, чтобы иметь дело с чистыми вопросами

In [49]:
df_clean = df.copy()
df_clean['question1'] = df.apply(lambda row: clean_sentence_lemmatizer(row['question1']), axis=1)
df_clean['question2'] = df.apply(lambda row: clean_sentence_lemmatizer(row['question2']), axis=1)
df_clean.head()

TypeError: ('expected string or bytes-like object', 'occurred at index 363362')